### SHOR'S ALGORITHM 

In [1]:
import qiskit
import matplotlib
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from  qiskit.tools.visualization  import circuit_drawer
from qiskit.extensions.standard import cx, cswap
from qiskit import BasicAer
import time
from qiskit.tools.visualization import plot_histogram
import numpy
from qiskit import IBMQ
IBMQ.enable_account("4750540e811a9918fc89afb3cf7899216e15b625a5973b40a2165b596b8c904729cdbeafa09d3924f5f71d60c5338cbc32ad4333a48812b27ca9c945ffa6092c","https://api.quantum-computing.ibm.com/api/Hubs/ibm-q/Groups/open/Projects/main")


In [2]:
def controlled_mult_2mod15_quantum(qr,qc,control_qubit):
    qc.cswap(control_qubit,qr[0],qr[3])# Swap 0th qubit and 3rd qubit
    qc.cswap(control_qubit,qr[1],qr[0])# Swap 0th qubit and 1st qubit
    qc.cswap(control_qubit,qr[1],qr[2])# Swap 1st qubit and 2nd qubit

In [3]:
import math
def shors_subroutine_period_2mod15(qr,qc,cr):
    qc.x(qr[0])
    qc.h(qr[4])
    qc.h(qr[4])
    qc.measure(qr[4],cr[0])

    qc.h(qr[5])
    qc.cx(qr[5],qr[0])
    qc.cx(qr[5],qr[2])
    if cr[0] == 1:
        qc.u1(math.pi/2,qr[4]) #pi/2
    qc.h(qr[5])
    qc.measure(qr[5],cr[1])

    qc.h(qr[6])
    controlled_mult_2mod15_quantum(qr,qc,qr[6])
    if cr[1] == 1:
        qc.u1(math.pi/2,qr[6]) # pi/2 
    if cr[0] == 1:
        qc.u1(math.pi/4,qr[6]) #pi/4 
    qc.h(qr[6])
    qc.measure(qr[6],cr[2])


In [4]:
import math
def period_from_quantum_measurement(quantum_measurement, number_qubits, a_shor, N_shor, max_steps=100): 
    xi=quantum_measurement/2**number_qubits
    all_as=[]
    all_xis=[]
    a_0=math.floor(xi)
    xi_0=xi-a_0
    all_as+=[a_0]
    all_xis+=[xi_0]
    all_ps=[]
    all_qs=[]
    p_0=all_as[0]
    q_0=1
    all_ps+=[p_0]
    all_qs+=[q_0]
    xi_n=xi_0
    #print(xi_n)
    while not numpy.isclose(xi_n,0,atol=1e-7):
        if len(all_as)>=max_steps:
            print("Warning: algorithm did not converge within max_steps %d steps, try increasing max_steps"%max_steps)
            break
        # computing a and xi
        a_nplus1=math.floor(1/xi_n)
        xi_nplus1=1/xi_n-a_nplus1
        all_as+=[a_nplus1]
        all_xis+=[xi_nplus1]
        xi_n=xi_nplus1
        n=len(all_as)-1
        if n==1:
            p_1=all_as[1]*all_as[0]+1
            q_1=all_as[1]
            all_ps+=[p_1]
            all_qs+=[q_1]
        else:
            p_n=all_as[n]*all_ps[n-1]+all_ps[n-2]
            q_n=all_as[n]*all_qs[n-1]+all_qs[n-2]
            all_ps+=[p_n]
            all_qs+=[q_n]
        
        if a_shor**all_qs[-1]%N_shor == 1 % N_shor:
            return all_qs[-1]

#period_from_quantum_measurement(13453,14,3,91)

In [5]:
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

def run_shors_subroutine_period2_mod15():
    qr = QuantumRegister(7)
    cr = ClassicalRegister(3)
    qc = QuantumCircuit(qr,cr)
    # run circuit (which includes measurement steps)
    shors_subroutine_period_2mod15(qr,qc,cr)
    print(qc)
    backend=BasicAer.get_backend('qasm_simulator')
    job_exp = qiskit.execute(qc, backend=backend,shots=1)
    result = job_exp.result()
    final=result.get_counts(qc)
    print(final)
    #print(int(list(final.keys())[0],2))
    #convert final result to decimal
    measurement=int(list(final.keys())[0],2)
    period_r=period_from_quantum_measurement(measurement,3,2,15)
    return period_r

#print(run_shors_subroutine_period2_mod15())

In [6]:
def period_finding_quantum(a,N):
    if a==2 and N==15:
        return run_shors_subroutine_period2_mod15()
    else:
        raise Exception("Not implemented for N=%d, a=%d" % (N,a))
        
def shors_algorithm_quantum(N,fixed_a=None):
    assert(N>0)
    assert(int(N)==N)
    while True:
        if not fixed_a:
            a=random.randint(0,N-1) 
        else:
            a=fixed_a
        g=math.gcd(a,N)
        if g!=1 or N==1:
            first_factor=g
            second_factor=int(N/g)
            return first_factor,second_factor
        else:
            r=period_finding_quantum(a,N)  
            if not r:
                continue
            if r % 2 != 0:
                continue
            elif a**(int(r/2)) % N == -1 % N:
                continue
            else:
                first_factor=math.gcd(a**int(r/2)+1,N)
                second_factor=math.gcd(a**int(r/2)-1,N)
                if first_factor==N or second_factor==N:
                    continue
                if first_factor*second_factor!=N:
                    continue
                return first_factor,second_factor

shors_algorithm_quantum(15,fixed_a=2)

         ┌───┐┌───┐                                      
q0_0: |0>┤ X ├┤ X ├──────────────X──X────────────────────
         └───┘└─┬─┘              │  │                    
q0_1: |0>───────┼────────────────┼──X──────────X─────────
                │          ┌───┐ │  │          │         
q0_2: |0>───────┼──────────┤ X ├─┼──┼──────────X─────────
                │          └─┬─┘ │  │          │         
q0_3: |0>───────┼────────────┼───X──┼──────────┼─────────
         ┌───┐  │  ┌───┐┌─┐  │   │  │          │         
q0_4: |0>┤ H ├──┼──┤ H ├┤M├──┼───┼──┼──────────┼─────────
         ├───┤  │  └───┘└╥┘  │   │  │ ┌───┐┌─┐ │         
q0_5: |0>┤ H ├──■────────╫───■───┼──┼─┤ H ├┤M├─┼─────────
         ├───┤           ║       │  │ └───┘└╥┘ │ ┌───┐┌─┐
q0_6: |0>┤ H ├───────────╫───────■──■───────╫──■─┤ H ├┤M├
         └───┘           ║                  ║    └───┘└╥┘
 c0_0: 0 ════════════════╩══════════════════╬══════════╬═
                                            ║          ║ 
 c0_1: 0 ═════

(5, 3)